#**Final Pipleline**

##Package Installation

In [1]:
#Package Installation
!pip install lxml
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q
!pip install peft -q
!pip install auto-gptq -q
!pip install optimum -q
!pip install bitsandbytes -q
!pip install wheel flash_attn -q
!pip install accelerate -q
!pip install requests -q
!pip install beautifulsoup4 -q
!pip install flash_attn==2.5.8 torch==2.3.1 accelerate==0.31.0 transformers==4.41.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 617.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

##Importing Packages

In [1]:
import requests
import torch
from bs4 import BeautifulSoup
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig, pipeline

##Web Scraping

CODEFORCES

In [2]:
#Web scraping
def scrape(url):
  vocab = {"<p>":" ", "</p>":" ", "$$$":'', "\\leq":" <= ", "\\le":" <= ", r'\xa0':" ", "\\cdot":" x ", "\\ldots":"...","\\dots":"...",
         '</span>':' ' , '<span class="tex-font-style-it">':' ', '<span class="tex-font-style-bf">':' ','<span class="tex-font-style-tt">':' ' ,
         '\\ne': '≠', r'\xa0':'', '\\oplus':' xor ', '\\,':'', '&lt':'<', '&gt':'>', "^\\dagger":'', '\\ge':' >= ', '\\operatorname':'',
         "\'":''}

  r = requests.get(url)

  # Parsing the HTML
  soup = BeautifulSoup(r.content, 'html.parser')

  s = soup.find('div', class_='entry-content')
  ques = soup.find('div',{"class":"problem-statement"}).get_text()
  input = soup.find('div',{"class":"input-specification"}).get_text()
  ouput = soup.find('div',{"class":"output-specification"}).get_text()

  ques = ques.split("Note")[0]
  ques = "".join(map(str, ques))
  ques = ques.split("standard output")[1]
  ques = "".join(map(str, ques))
  ques = ques.split("ExamplesInput")[0]
  ques = "".join(map(str, ques))
  ques = ques.split("ExampleInput")[0]

  for word in vocab:
    lst = []
    for i in ques.split(word):
      if i!='':
        lst.append(i)
    # lst
    ques = vocab[word].join(map(str, lst))
  return ques

LEETCODE

In [3]:
def scrape_leetcode(problem_name):
  data = {"operationName":"questionData","variables":{"titleSlug":f"{problem_name}"},"query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    langToValidPlayground\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    envInfo\n    libraryUrl\n    __typename\n  }\n}\n"}

  r = requests.post('https://leetcode.com/graphql', json = data).json()
  soup = bs(r['data']['question']['content'], 'lxml')
  title = r['data']['question']['title']
  question =  soup.get_text().replace('\n',' ')

  question_p1 = question.split('Example')
  question_p2 = question.split('Constraint')
  ques = [question_p1[0], 'Constraint' + question_p2[1]]
  ques = ' '.join(map(str, ques))
  ques = ques.split('\xa0')
  ques = ''.join(map(str, ques))
  return ques

##Scraping the question from user intput

In [4]:
Platform = input("Enter L for LeetCode and C for Codeforces)")
if Platform == "C":
  problem_number = input("Enter Contest Id:")
  part = input('Enter Question No.:')

  url = r"https://codeforces.com/problemset/problem/" + f"{problem_number}" + r"/" + f"{part}"

  ques = scrape(url)
elif Platform == "L":
  problem_name = input("Enter Problem Name:")
  ques = scrape_leetcode(problem_name)

else:
  print("Invalid Platform")

Enter L for LeetCode and C for Codeforces)C
Enter Contest Id:1709
Enter Question No.:B


AttributeError: 'NoneType' object has no attribute 'get_text'

In [5]:
ques = "Write an algorithm to make arryay of size n sorted in ascedning order"

##Making a Good Prompt

###LLM help LLM

In [6]:
torch.random.manual_seed(0)
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    quantization_config=config,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": '''Can you provide an elaborate explanation of the following programming based question? One hot summer day Pete and his friend Billy decided to buy a watermelon. They chose the biggest and the ripest one, in their opinion. After that the watermelon was weighed, and the scales showed w kilos. They rushed home, dying of thirst, and decided to divide the berry, however they faced a hard problem.
Pete and Billy are great fans of even numbers, that's why they want to divide the watermelon in such a way that each of the two parts weighs even number of kilos, at the same time it is not obligatory that the parts are equal. The boys are extremely tired and want to start their meal as soon as possible, that's why you should help them and find out, if they can divide the watermelon in the way they want. For sure, each of them should get a part of positive weight.'''},
    {"role": "assistant", "content": '''Certainly! Let’s break down the problem and provide an elaborate explanation:

Problem Statement
Pete and Billy have a watermelon that weighs w kilos. They want to divide it into two parts such that:

Each part weighs an even number of kilos.
The parts do not necessarily have to be equal in weight.
Both Pete and Billy should receive a part with a positive weight.
The goal is to determine whether it’s possible to divide the watermelon according to these conditions.

Approach
To solve this problem, we need to consider a few observations:

If the total weight of the watermelon (w) is odd, it cannot be divided into two even parts. In this case, the answer is “NO.”
If the total weight of the watermelon (w) is even, we can always divide it into two equal parts (each with weight w/2), which satisfies the conditions.
Detailed Explanation
Odd Total Weight (w is odd):
If w is odd, it cannot be divided into two even parts.
An odd number cannot be expressed as the sum of two even numbers.
Therefore, the answer is “NO.”
Even Total Weight (w is even):
If w is even, we can divide it into two equal parts, each with weight w/2.
Since w/2 is always even, both Pete and Billy will receive even-weighted parts.
The answer is “YES.”
Summary
If the total weight of the watermelon is odd, it cannot be divided as desired.
If the total weight of the watermelon is even, it can always be divided into two even-weighted parts.
Example
Suppose the watermelon weighs 10 kilos (w = 10):

Pete and Billy can each receive a 5-kilo part, which satisfies the conditions.
The answer is “YES.”
Remember that this problem is a simple observation-based solution. No complex calculations or algorithms are needed. We just need to check whether the total weight is even or odd. '''},
    {"role": "user", "content": f'''please elaborate the following too : {ques}'''},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 1000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Certainly! Let's break down the problem and provide an elaborate explanation:

Problem Statement
You are given an unsorted array of size n. The goal is to write an algorithm that sorts the array in ascending order.

Approach
To solve this problem, we can use a sorting algorithm. There are various sorting algorithms available, such as Bubble Sort, Selection Sort, Insertion Sort, Merge Sort, Quick Sort, and Heap Sort. For this explanation, we will use the Merge Sort algorithm, which is a divide-and-conquer algorithm that has a time complexity of O(n log n).

Detailed Explanation
Merge Sort Algorithm:

1. Divide the unsorted array into two halves (left and right).
2. Recursively sort each half using the Merge Sort algorithm.
3. Merge the two sorted halves into a single sorted array.

Here's the step-by-step process of the Merge Sort algorithm:

1. Divide the unsorted array into two halves (left and right).

2. Recursively sort each half using the Merge Sort algorithm.

   a. If the size 

In [7]:
context = output[0]['generated_text']

In [8]:
torch.cuda.empty_cache()

In [9]:
prompt_template_w_context = lambda context, comment: f"""You always output C++ code for the problem asked to you

{context}
Please give code for the following question in C++ also give the explanation for your answer. Use the context above if it is helpful.

{ques}

"""

prompt = prompt_template_w_context(context, ques)


##Loading Model

In [10]:
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-Coder-V2-Lite-Base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-Coder-V2-Lite-Base", quantization_config=config, trust_remote_code=True,torch_dtype=torch.bfloat16)
input_text = "#write a quick sort algorithm"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=1000)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
model = torch.compile(model, mode="reduce-overhead", fullgraph=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


#write a quick sort algorithm

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less = [i for i in arr[1:] if i <= pivot]
        greater = [i for i in arr[1:] if i > pivot]
        return quick_sort(less) + [pivot] + quick_sort(greater)

print(quick_sort([3, 6, 8, 10, 1, 2, 1]))



##Taking Inference

In [13]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=1500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


You always output C++ code for the problem asked to you

 Certainly! Let's break down the problem and provide an elaborate explanation:

Problem Statement
You are given an unsorted array of size n. The goal is to write an algorithm that sorts the array in ascending order.

Approach
To solve this problem, we can use a sorting algorithm. There are various sorting algorithms available, such as Bubble Sort, Selection Sort, Insertion Sort, Merge Sort, Quick Sort, and Heap Sort. For this explanation, we will use the Merge Sort algorithm, which is a divide-and-conquer algorithm that has a time complexity of O(n log n).

Detailed Explanation
Merge Sort Algorithm:

1. Divide the unsorted array into two halves (left and right).
2. Recursively sort each half using the Merge Sort algorithm.
3. Merge the two sorted halves into a single sorted array.

Here's the step-by-step process of the Merge Sort algorithm:

1. Divide the unsorted array into two halves (left and right).

2. Recursively sort each

##Recommentation System

In [ ]:
import requests
from bs4 import BeautifulSoup
Tags = ["strings"]
Rating  = int(input("Enter your current rating:"))
for Tag in Tags:
  url = f"https://codeforces.com/problemset?tags={Tag},{Rating+100}-{Rating+200}"
  r = requests.get(url)
  soup = BeautifulSoup(r.content, 'html.parser')
  elems = soup.find_all('div',style="float: left;")
  i = 0
  for elem in elems:
    anchor_tag = elem.find('a')
    href_value = anchor_tag.get('href')
    print(f"https://codeforces.com{href_value}")
    i += 1
    if i == 5:
      break